# Chapter 16 – Natural Language Processing with RNNs and Attention

This notebook contains all the sample code and solutions to the exercises in chapter 16.

## Setup
This project requires Python 3.7 or above:

In [3]:
import sys

assert sys.version_info >= (3, 7)

*Warning*: the latest TensorFlow versions are based on Keras 3. For previous chapters, it wasn't too hard to update the code to support Keras 3, but unfortunately it's much harder for this chapter: for example, stateful RNNs work very differently, ragged tensors are no longer supported, TensorFlow Hub models are no longer supported, and more. So for this chapter I've had to revert to Keras 2. To do that, I set the TF_USE_LEGACY_KERAS environment variable to "1" and import the tf_keras package. This ensures that tf.keras points to tf_keras, which is Keras 2.*.

In [4]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1" 

# import tf_keras

In [5]:
from packaging import version
import tensorflow as tf

assert version.parse(tf.__version__) >= version.parse("2.8.0")

As we did in earlier chapters, let's define the default font sizes to make the figures prettier:

In [6]:
import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

And let's create the ```images/nlp``` folder (if it doesn't already exist), and define the ```save_fig()``` function which is used through this notebook to save the figures in high-res for the book:

In [7]:
from pathlib import Path

IMAGES_PATH = Path() / "images" / "nlp"
IMAGES_PATH.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

This chapter can be very slow without a GPU, so let's make sure there's one, or else issue a warning:

In [8]:
if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. Neural nets can be very slow without a GPU.")
    if "google.colab" in sys.modules:
        print("Go to Runtime > Change runtime and select a GPU hardware "
              "accelerator.")
    if "kaggle_secrets" in sys.modules:
        print("Go to Settings > Accelerator and select GPU.")

# Generating Shakespearean Text Using a Character RNN
## Creating the Training Dataset
Let's download the Shakespeare data from Andrej Karpathy's char-rnn project

In [9]:
import tensorflow as tf

shakespeare_url = "https://homl.info/shakespeare"  # shortcut URL
filepath = tf.keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [10]:
# extra code – shows a short text sample
print(shakespeare_text[:80])


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.


In [11]:
# extra code – shows all 39 distinct characters (after converting to lower case)
"".join(sorted(set(shakespeare_text.lower())))

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

In [12]:
text_vec_layer = tf.keras.layers.TextVectorization(split="character",
                                                   standardize="lower")
text_vec_layer.adapt([shakespeare_text])
encoded = text_vec_layer([shakespeare_text])[0]

In [13]:
encoded -= 2  # drop tokens 0 (pad) and 1 (unknown), which we will not use
n_tokens = text_vec_layer.vocabulary_size() - 2  # number of distinct chars = 39
dataset_size = len(encoded)  # total number of chars = 1,115,394

In [14]:
n_tokens

39

In [15]:
dataset_size

1115394

In [16]:
def to_dataset(sequence, length, shuffle=False, seed=None, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda window_ds: window_ds.batch(length + 1))
    if shuffle:
        ds = ds.shuffle(100_000, seed=seed)
    ds = ds.batch(batch_size)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

In [17]:
# extra code – a simple example using to_dataset()
# There's just one sample in this dataset: the input represents "to b" and the
# output represents "o be"
list(to_dataset(text_vec_layer(["To be"])[0], length=4))

[(<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[ 4,  5,  2, 23]], dtype=int64)>,
  <tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[ 5,  2, 23,  3]], dtype=int64)>)]

In [18]:
length = 100
tf.random.set_seed(42)
train_set = to_dataset(encoded[:1_000_000], length=length, shuffle=True,
                       seed=42)
valid_set = to_dataset(encoded[1_000_000:1_060_000], length=length)
test_set = to_dataset(encoded[1_060_000:], length=length)

## Building and Training the Char-RNN Model
Warning: the following code may one or two hours to run, depending on your GPU. Without a GPU, it may take over 24 hours. If you don't want to wait, just skip the next two code cells and run the code below to download a pretrained model.

Note: the GRU class will only use cuDNN acceleration (assuming you have a GPU) when using the default values for the following arguments: activation, recurrent_activation, recurrent_dropout, unroll, use_bias and reset_after.

In [23]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])

In [19]:
# tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU
# model = tf.keras.Sequential([
#     tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.Dense(n_tokens, activation="softmax")
# ])
# model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
#               metrics=["accuracy"])
# model_ckpt = tf.keras.callbacks.ModelCheckpoint(
#     "my_shakespeare_model", monitor="val_accuracy", save_best_only=True)
# history = model.fit(train_set, validation_data=valid_set, epochs=10,
#                     callbacks=[model_ckpt])

In [24]:
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X - 2),  # no <PAD> or <UNK> tokens
    model
])

If you don't want to wait for training to complete, I've pretrained a model for you. The following code will download it. Uncomment the last line if you want to use it instead of the model trained above.

In [25]:
# extra code – downloads a pretrained model
url = "https://github.com/ageron/data/raw/main/shakespeare_model.tgz"
path = tf.keras.utils.get_file("shakespeare_model.tgz", url, extract=True)
model_path = Path(path).with_name("shakespeare_model")
shakespeare_model = tf.keras.models.load_model(model_path)

In [26]:
y_proba = shakespeare_model.predict(["To be or not to b"])[0, -1]
y_pred = tf.argmax(y_proba)  # choose the most probable character ID
text_vec_layer.get_vocabulary()[y_pred + 2]

1/1 [==============================] - 3s 3s/step


'e'

## Generating Fake Shakespearean Text

In [27]:
log_probas = tf.math.log([[0.5, 0.4, 0.1]])  # probas = 50%, 40%, and 10%
tf.random.set_seed(42)
tf.random.categorical(log_probas, num_samples=8)  # draw 8 samples

<tf.Tensor: shape=(1, 8), dtype=int64, numpy=array([[0, 0, 1, 1, 1, 0, 0, 0]], dtype=int64)>

In [28]:
def next_char(text, temperature=1):
    y_proba = shakespeare_model.predict([text])[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0, 0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

In [29]:
def extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [30]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU

In [31]:
print(extend_text("To be or not to be", temperature=0.01))

1/1 [==============================] - 0s 26ms/step
To be or not to be the duke
as it is a proper strange death,
and the


In [32]:
print(extend_text("To be or not to be", temperature=1))

1/1 [==============================] - 0s 25ms/step
To be or not to begg, i
point and through you are to-morrow?

romeo:


In [33]:
print(extend_text("To be or not to be", temperature=100))

1/1 [==============================] - 0s 26ms/step
To be or not to bepevicm-vilv!?$mz?gmjz :3?ljb'va;!td&
i.ur3l'-j!3em


## Stateful RNN

In [34]:
def to_dataset_for_stateful_rnn(sequence, length):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=length, drop_remainder=True)
    ds = ds.flat_map(lambda window: window.batch(length + 1)).batch(1)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

stateful_train_set = to_dataset_for_stateful_rnn(encoded[:1_000_000], length)
stateful_valid_set = to_dataset_for_stateful_rnn(encoded[1_000_000:1_060_000],
                                                 length)
stateful_test_set = to_dataset_for_stateful_rnn(encoded[1_060_000:], length)

In [35]:
# extra code – simple example using to_dataset_for_stateful_rnn()
list(to_dataset_for_stateful_rnn(tf.range(10), 3))

[(<tf.Tensor: shape=(1, 3), dtype=int32, numpy=array([[0, 1, 2]])>,
  <tf.Tensor: shape=(1, 3), dtype=int32, numpy=array([[1, 2, 3]])>),
 (<tf.Tensor: shape=(1, 3), dtype=int32, numpy=array([[3, 4, 5]])>,
  <tf.Tensor: shape=(1, 3), dtype=int32, numpy=array([[4, 5, 6]])>),
 (<tf.Tensor: shape=(1, 3), dtype=int32, numpy=array([[6, 7, 8]])>,
  <tf.Tensor: shape=(1, 3), dtype=int32, numpy=array([[7, 8, 9]])>)]

If you'd like to have more than one window per batch, you can use the ```to_batched_dataset_for_stateful_rnn()``` function instead of ```to_dataset_for_stateful_rnn()```:

In [36]:
# extra code – shows one way to prepare a batched dataset for a stateful RNN

import numpy as np

def to_non_overlapping_windows(sequence, length):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=length, drop_remainder=True)
    return ds.flat_map(lambda window: window.batch(length + 1))

def to_batched_dataset_for_stateful_rnn(sequence, length, batch_size=32):
    parts = np.array_split(sequence, batch_size)
    datasets = tuple(to_non_overlapping_windows(part, length) for part in parts)
    ds = tf.data.Dataset.zip(datasets).map(lambda *windows: tf.stack(windows))
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

list(to_batched_dataset_for_stateful_rnn(tf.range(20), length=3, batch_size=2))

[(<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 0,  1,  2],
         [10, 11, 12]])>,
  <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 1,  2,  3],
         [11, 12, 13]])>),
 (<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 3,  4,  5],
         [13, 14, 15]])>,
  <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 4,  5,  6],
         [14, 15, 16]])>),
 (<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 6,  7,  8],
         [16, 17, 18]])>,
  <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 7,  8,  9],
         [17, 18, 19]])>)]

In [37]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16,
                              batch_input_shape=[1, None]),
    tf.keras.layers.GRU(128, return_sequences=True, stateful=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])


In [38]:
class ResetStatesCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [39]:
# extra code – use a different directory to save the checkpoints
model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    "my_stateful_shakespeare_model",
    monitor="val_accuracy",
    save_best_only=True)

**Warning**: the following cell will take a while to run (possibly an hour if you are not using a GPU).

In [40]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(stateful_train_set, validation_data=stateful_valid_set,
                    epochs=10, callbacks=[ResetStatesCallback(), model_ckpt])

Epoch 1/10
   9994/Unknown - 64s 6ms/step - loss: 1.8660 - accuracy: 0.4507

INFO:tensorflow:Assets written to: my_stateful_shakespeare_model\assets


INFO:tensorflow:Assets written to: my_stateful_shakespeare_model\assets


9999/9999 [==============================] - 67s 7ms/step - loss: 1.8658 - accuracy: 0.4507 - val_loss: 1.7152 - val_accuracy: 0.4884
Epoch 2/10
9992/9999 [============================>.] - ETA: 0s - loss: 1.5634 - accuracy: 0.5277

INFO:tensorflow:Assets written to: my_stateful_shakespeare_model\assets


...

INFO:tensorflow:Assets written to: my_stateful_shakespeare_model\assets


9999/9999 [==============================] - 66s 7ms/step - loss: 1.3825 - accuracy: 0.5745 - val_loss: 1.5397 - val_accuracy: 0.5364
Epoch 9/10
9999/9999 [==============================] - 63s 6ms/step - loss: 1.3756 - accuracy: 0.5762 - val_loss: 1.5374 - val_accuracy: 0.5356
Epoch 10/10
9993/9999 [============================>.] - ETA: 0s - loss: 1.3705 - accuracy: 0.5774

INFO:tensorflow:Assets written to: my_stateful_shakespeare_model\assets


INFO:tensorflow:Assets written to: my_stateful_shakespeare_model\assets


9999/9999 [==============================] - 65s 6ms/step - loss: 1.3704 - accuracy: 0.5774 - val_loss: 1.5300 - val_accuracy: 0.5395


**Extra Material: converting the stateful RNN to a stateless RNN and using it**

To use the model with different batch sizes, we need to create a stateless copy:

In [41]:
stateless_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])

To set the weights, we first need to build the model (so the weights get created):

In [42]:
stateless_model.build(tf.TensorShape([None, None]))

In [43]:
stateless_model.set_weights(model.get_weights())

In [44]:
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X - 2),  # no <PAD> or <UNK> tokens
    stateless_model
])

In [45]:
tf.random.set_seed(42)

print(extend_text("to be or not to be", temperature=0.01))

1/1 [==============================] - 0s 20ms/step
to be or not to be
than a great in a great in a great in the world,



## Sentiment Analysis

In [46]:
import tensorflow_datasets as tfds

raw_train_set, raw_valid_set, raw_test_set = tfds.load(
    name="imdb_reviews",
    split=["train[:90%]", "train[90%:]", "test"],
    as_supervised=True
)
tf.random.set_seed(42)
train_set = raw_train_set.shuffle(5000, seed=42).batch(32).prefetch(1)
valid_set = raw_valid_set.batch(32).prefetch(1)
test_set = raw_test_set.batch(32).prefetch(1)

In [47]:
for review, label in raw_train_set.take(4):
    print(review.numpy().decode("utf-8")[:200], "...")
    print("Label:", label.numpy())

This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting  ...
Label: 0
I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However  ...
Label: 0
Mann photographs the Alberta Rocky Mountains in a superb fashion, and Jimmy Stewart and Walter Brennan give enjoyable performances as they always seem to do. <br /><br />But come on Hollywood - a Moun ...
Label: 0
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful perf ...
Label: 1


In [48]:
vocab_size = 1000
text_vec_layer = tf.keras.layers.TextVectorization(max_tokens=vocab_size)
text_vec_layer.adapt(train_set.map(lambda reviews, labels: reviews))

**Warning**: the following cell will take a few minutes to run and the model will probably not learn anything because we didn't mask the padding tokens (that's the point of the next section).

In [49]:
embed_size = 128
tf.random.set_seed(42)
model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Embedding(vocab_size, embed_size),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=2)

Epoch 1/2
704/704 [==============================] - 20s 27ms/step - loss: 0.6934 - accuracy: 0.4988 - val_loss: 0.6931 - val_accuracy: 0.5012
Epoch 2/2
704/704 [==============================] - 19s 27ms/step - loss: 0.6958 - accuracy: 0.5005 - val_loss: 0.7236 - val_accuracy: 0.5028


## Masking
**Warning**: the following cell will take a while to run (possibly 30 minutes if you are not using a GPU).

In [50]:
embed_size = 128
tf.random.set_seed(42)
model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Embedding(vocab_size, embed_size, mask_zero=True),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=5)

Epoch 1/5
704/704 [==============================] - 23s 30ms/step - loss: 0.5528 - accuracy: 0.7124 - val_loss: 0.3743 - val_accuracy: 0.8404
Epoch 2/5
704/704 [==============================] - 20s 29ms/step - loss: 0.3387 - accuracy: 0.8529 - val_loss: 0.3065 - val_accuracy: 0.8708
Epoch 3/5
704/704 [==============================] - 20s 28ms/step - loss: 0.2918 - accuracy: 0.8781 - val_loss: 0.3177 - val_accuracy: 0.8616
Epoch 4/5
704/704 [==============================] - 20s 29ms/step - loss: 0.2703 - accuracy: 0.8888 - val_loss: 0.3012 - val_accuracy: 0.8740
Epoch 5/5
704/704 [==============================] - 21s 30ms/step - loss: 0.2534 - accuracy: 0.8978 - val_loss: 0.3120 - val_accuracy: 0.8584


Or using manual masking:

In [51]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on the CPU
inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)
token_ids = text_vec_layer(inputs)
mask = tf.math.not_equal(token_ids, 0)
Z = tf.keras.layers.Embedding(vocab_size, embed_size)(token_ids)
Z = tf.keras.layers.GRU(128, dropout=0.2)(Z, mask=mask)
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(Z)
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])

Warning: the following cell will take a while to run (possibly 30 minutes if you are not using a GPU).

In [52]:
# extra code – compiles and trains the model, as usual
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=5)

Epoch 1/5


704/704 [==============================] - 23s 30ms/step - loss: 0.5597 - accuracy: 0.7056 - val_loss: 0.6230 - val_accuracy: 0.5940
Epoch 2/5
704/704 [==============================] - 20s 29ms/step - loss: 0.3712 - accuracy: 0.8381 - val_loss: 0.3165 - val_accuracy: 0.8680
Epoch 3/5
704/704 [==============================] - 20s 29ms/step - loss: 0.2979 - accuracy: 0.8748 - val_loss: 0.3250 - val_accuracy: 0.8600
Epoch 4/5
704/704 [==============================] - 20s 29ms/step - loss: 0.2751 - accuracy: 0.8874 - val_loss: 0.3076 - val_accuracy: 0.8676
Epoch 5/5
704/704 [==============================] - 20s 29ms/step - loss: 0.2605 - accuracy: 0.8933 - val_loss: 0.2984 - val_accuracy: 0.8676


**Extra material: using ragged tensors**

In [53]:
text_vec_layer_ragged = tf.keras.layers.TextVectorization(
    max_tokens=vocab_size, ragged=True)
text_vec_layer_ragged.adapt(train_set.map(lambda reviews, labels: reviews))
text_vec_layer_ragged(["Great movie!", "This is DiCaprio's best role."])

<tf.RaggedTensor [[86, 18], [11, 7, 1, 116, 217]]>

In [54]:
text_vec_layer(["Great movie!", "This is DiCaprio's best role."])

<tf.Tensor: shape=(2, 5), dtype=int64, numpy=
array([[ 86,  18,   0,   0,   0],
       [ 11,   7,   1, 116, 217]], dtype=int64)>

**Warning**: the following cell will take a while to run (possibly 30 minutes if you are not using a GPU).

In [55]:
embed_size = 128
tf.random.set_seed(42)
model = tf.keras.Sequential([
    text_vec_layer_ragged,
    tf.keras.layers.Embedding(vocab_size, embed_size),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=5)

Epoch 1/5


c:\Users\schre\anaconda3\envs\tf_dev\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_7/gru_6/RaggedToTensor/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_7/gru_6/RaggedToTensor/boolean_mask/GatherV2:0", shape=(None, 128), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_7/gru_6/RaggedToTensor/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


704/704 [==============================] - 21s 29ms/step - loss: 0.5799 - accuracy: 0.6938 - val_loss: 0.5038 - val_accuracy: 0.7572
Epoch 2/5
704/704 [==============================] - 20s 29ms/step - loss: 0.3641 - accuracy: 0.8420 - val_loss: 0.3391 - val_accuracy: 0.8452
Epoch 3/5
704/704 [==============================] - 20s 29ms/step - loss: 0.3130 - accuracy: 0.8672 - val_loss: 0.4306 - val_accuracy: 0.8112
Epoch 4/5
704/704 [==============================] - 21s 29ms/step - loss: 0.2882 - accuracy: 0.8806 - val_loss: 0.3529 - val_accuracy: 0.8472
Epoch 5/5
704/704 [==============================] - 21s 30ms/step - loss: 0.2707 - accuracy: 0.8884 - val_loss: 0.3659 - val_accuracy: 0.8308


## Reusing Pretrained Embeddings and Language Models
**Warning**: the following cell will take a while to run (possibly an hour if you are not using a GPU).

In [56]:
import os
import tensorflow_hub as hub

os.environ["TFHUB_CACHE_DIR"] = "my_tfhub_cache"
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU
model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                   trainable=True, dtype=tf.string, input_shape=[]),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit(train_set, validation_data=valid_set, epochs=10)

Epoch 1/10
704/704 [==============================] - 60s 74ms/step - loss: 0.3149 - accuracy: 0.8662 - val_loss: 0.2436 - val_accuracy: 0.8992
Epoch 2/10
704/704 [==============================] - 52s 74ms/step - loss: 0.0504 - accuracy: 0.9841 - val_loss: 0.3876 - val_accuracy: 0.8796
Epoch 3/10
704/704 [==============================] - 52s 74ms/step - loss: 0.0066 - accuracy: 0.9988 - val_loss: 0.4070 - val_accuracy: 0.8940
Epoch 4/10
704/704 [==============================] - 52s 74ms/step - loss: 0.0012 - accuracy: 0.9999 - val_loss: 0.4652 - val_accuracy: 0.8936
Epoch 5/10
704/704 [==============================] - 52s 74ms/step - loss: 4.9306e-04 - accuracy: 1.0000 - val_loss: 0.4800 - val_accuracy: 0.8944
Epoch 6/10
704/704 [==============================] - 52s 74ms/step - loss: 4.8683e-04 - accuracy: 1.0000 - val_loss: 0.4879 - val_accuracy: 0.8948
Epoch 7/10
704/704 [==============================] - 52s 73ms/step - loss: 4.1655e-04 - accuracy: 1.0000 - val_loss: 0.4899 - v

## An Encoder–Decoder Network for Neural Machine Translation

In [57]:

url = "https://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip"
path = tf.keras.utils.get_file("spa-eng.zip", origin=url, cache_dir="datasets",
                               extract=True)
text = (Path(path).with_name("spa-eng") / "spa.txt").read_text()

In [58]:
import numpy as np

text = text.replace("¡", "").replace("¿", "")
pairs = [line.split("\t") for line in text.splitlines()]
np.random.seed(42)  # extra code – ensures reproducibility on CPU
np.random.shuffle(pairs)
sentences_en, sentences_es = zip(*pairs)  # separates the pairs into 2 lists

In [59]:
for i in range(3):
    print(sentences_en[i], "=>", sentences_es[i])

How boring! => Qué aburrimiento!
I love sports. => Adoro el deporte.
Would you like to swap jobs? => Te gustaría que intercambiemos los trabajos?


In [60]:
vocab_size = 1000
max_length = 50
text_vec_layer_en = tf.keras.layers.TextVectorization(
    vocab_size, output_sequence_length=max_length)
text_vec_layer_es = tf.keras.layers.TextVectorization(
    vocab_size, output_sequence_length=max_length)
text_vec_layer_en.adapt(sentences_en)
text_vec_layer_es.adapt([f"startofseq {s} endofseq" for s in sentences_es])

In [61]:

text_vec_layer_en.get_vocabulary()[:10]

['', '[UNK]', 'the', 'i', 'to', 'you', 'tom', 'a', 'is', 'he']

In [62]:
text_vec_layer_es.get_vocabulary()[:10]

['', '[UNK]', 'startofseq', 'endofseq', 'de', 'que', 'a', 'no', 'tom', 'la']

In [63]:
X_train = tf.constant(sentences_en[:100_000])
X_valid = tf.constant(sentences_en[100_000:])
X_train_dec = tf.constant([f"startofseq {s}" for s in sentences_es[:100_000]])
X_valid_dec = tf.constant([f"startofseq {s}" for s in sentences_es[100_000:]])
Y_train = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[:100_000]])
Y_valid = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[100_000:]])

In [64]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU
encoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)
decoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)

In [65]:
embed_size = 128
encoder_input_ids = text_vec_layer_en(encoder_inputs)
decoder_input_ids = text_vec_layer_es(decoder_inputs)
encoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size,
                                                    mask_zero=True)
decoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size,
                                                    mask_zero=True)
encoder_embeddings = encoder_embedding_layer(encoder_input_ids)
decoder_embeddings = decoder_embedding_layer(decoder_input_ids)

In [66]:
encoder = tf.keras.layers.LSTM(512, return_state=True)
encoder_outputs, *encoder_state = encoder(encoder_embeddings)

In [67]:
decoder = tf.keras.layers.LSTM(512, return_sequences=True)
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)

In [68]:
output_layer = tf.keras.layers.Dense(vocab_size, activation="softmax")
Y_proba = output_layer(decoder_outputs)

**Warning**: the following cell will take a while to run (possibly a couple hours if you are not using a GPU).

In [69]:
model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs],
                       outputs=[Y_proba])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit((X_train, X_train_dec), Y_train, epochs=10,
          validation_data=((X_valid, X_valid_dec), Y_valid))

Epoch 1/10
3125/3125 [==============================] - 34s 10ms/step - loss: 0.4108 - accuracy: 0.4305 - val_loss: 0.3045 - val_accuracy: 0.5280
Epoch 2/10
3125/3125 [==============================] - 32s 10ms/step - loss: 0.2621 - accuracy: 0.5763 - val_loss: 0.2350 - val_accuracy: 0.6084
Epoch 3/10
3125/3125 [==============================] - 32s 10ms/step - loss: 0.2048 - accuracy: 0.6476 - val_loss: 0.2037 - val_accuracy: 0.6532
Epoch 4/10
3125/3125 [==============================] - 30s 10ms/step - loss: 0.1721 - accuracy: 0.6930 - val_loss: 0.1902 - val_accuracy: 0.6719
Epoch 5/10
3125/3125 [==============================] - 31s 10ms/step - loss: 0.1490 - accuracy: 0.7260 - val_loss: 0.1848 - val_accuracy: 0.6791
Epoch 6/10
3125/3125 [==============================] - 33s 10ms/step - loss: 0.1304 - accuracy: 0.7545 - val_loss: 0.1832 - val_accuracy: 0.6836
Epoch 7/10
3125/3125 [==============================] - 31s 10ms/step - loss: 0.1143 - accuracy: 0.7786 - val_loss: 0.1854 -

In [70]:
def translate(sentence_en):
    translation = ""
    for word_idx in range(max_length):
        X = np.array([sentence_en])  # encoder input 
        X_dec = np.array(["startofseq " + translation])  # decoder input
        y_proba = model.predict((X, X_dec))[0, word_idx]  # last token's probas
        predicted_word_id = np.argmax(y_proba)
        predicted_word = text_vec_layer_es.get_vocabulary()[predicted_word_id]
        if predicted_word == "endofseq":
            break
        translation += " " + predicted_word
    return translation.strip()

In [71]:
translate("I like soccer")

1/1 [==============================] - 0s 11ms/step


'me gusta el fútbol'

In [72]:
translate("I like soccer and also going to the beach")

1/1 [==============================] - 0s 12ms/step


'me gusta jugar al tenis pero [UNK] de casa'

Nice! However, the model struggles with longer sentences: